In [2]:
import os

In [3]:
%pwd

'c:\\Users\\sujee\\Desktop\\Project\\ML-Project-With-ML-Flow\\Notebooks'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\sujee\\Desktop\\Project\\ML-Project-With-ML-Flow'

In [6]:
# entity file

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH): 

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir    
        )
        return data_ingestion_config

In [12]:
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # function to download the data and save it into local directory
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} file has been downloaded having the info as: {headers}")
        else:
            logger.info(f"{filename} already exists of size (KB): {get_size(Path(self.config.local_data_file))}")

    # extracting the zip file
    def extract_zip_file(self):
        """
            zip_file_path: str
            Extract zip file into data directory
            Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)

In [14]:
# updating pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-15 20:07:14,615: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-12-15 20:07:14,636: INFO: common: yaml file params.yaml loaded successfully]
[2024-12-15 20:07:14,641: INFO: common: yaml file schema.yaml loaded successfully]
[2024-12-15 20:07:14,647: INFO: common: created directory at artifacts]
[2024-12-15 20:07:14,652: INFO: common: created directory at artifacts/data_ingestion]
[2024-12-15 20:07:16,224: INFO: 2250710294: artifacts/data_ingestion/data.zip file has been downloaded having the info as: Connection: close
Content-Length: 20765
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9f8d93e73fd3cab98f6407673c3c12c1be7cf3174649275e9ce12f1787a9c21e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1F0F:42FD4:27F867:3256F9:675EE99A
Accept-Ranges: bytes
Date: 